# Documentation of the requests for the bso3-harvest-datacite application

In [2]:
import requests

payload = {}
# base_url = "http://127.0.0.1:5004/"
base_url = "http://datacite.bso3.dataesr.ovh/"
harvest_endpoint = "harvest_dois"
process_endpoint = "process"
affiliations_endpoint = "affiliations"
consolidate_affiliation_endpoint = "consolidate_affiliations_files"
enrichment_endpoint = "enrich_dois"
es_index_endpoint = "create_index"

harvest_url = base_url + harvest_endpoint
process_url = base_url + process_endpoint
affiliations_url = base_url + affiliations_endpoint
consolidate_affiliation_url = base_url + consolidate_affiliation_endpoint
enrichment_url = base_url + enrichment_endpoint
es_index_url = base_url + es_index_endpoint
headers = {"Content-Type": "application/json"}

In [2]:
date_prefix = "20221021"
date_prefix = "20230413"

## Harvest request
>Harvest the datacite API data via [dcdump](https://github.com/miku/dcdump)

In [4]:
harvest_payload = {
#    "target_directory": , # optional, default: RAW_DUMP_FOLDER_NAME (env)
#    "start_date": , # optional, default: DEFAULT_START_DATE ("2018-01-01")
#    "end_date": , # optional, default: current date
#    "interval": , # optional, default: day
#    "use_threads" : , # optional, default: False
#    "force" # optional, default: True
}
requests.post(harvest_url, headers=headers, json=harvest_payload)

<Response [202]>

## Process request
>Extraction of the info to submit to the affiliation matcher service from harvested datacite files

In [ ]:
process_payload = {
    "total_number_of_partitions": 100, # optional, default: 100 
    "file_prefix": date_prefix
}
requests.post(process_url, headers=headers, json=process_payload)

## Affiliation request
>Get affiliations via Affiliation Matcher for the data extracted during the process task

In [3]:
affiliations_payload = {
    "number_of_partitions": 100, # optional, default: 100
    "file_prefix": date_prefix
}
requests.post(affiliations_url, headers=headers, json=affiliations_payload)

<Response [202]>

## Enrichment request
>Take raw dcdump files and write a file for each doi containing affiliations informations. Upload them to OVH object storage, in the datacite bucket under /dois (all files) and /fr (copy of doi files containing fr affiliations). Prepare the Elastic search index source file.

In [3]:
index_name = f"bso3-datacite-{date_prefix}"
enrichment_payload = {
    # 1650 files
    "partition_size": 10, # optional, default: 10 
    "index_name": index_name
    # Cannot be much larger because of the memory
}
requests.post(enrichment_url, headers=headers, json=enrichment_payload)

<Response [202]>

## Create index ES request
>Run elasticdump against the Elastic Search index source file.

In [3]:
es_index_payload = {
    "index_name": index_name
}
requests.post(es_index_url, headers=headers, json=es_index_payload)

<Response [202]>